# Find Sublet Through Facebook Groups

There are three Fb groups that are mainly used by Waterloo students to find for a place to stay for 4-months in Kitchener-Waterloo region:
      
      a) "Student Housing in Waterloo": "https://www.facebook.com/groups/110354088989367/"
      b) "UW/WLU 4 Month Subletting": "https://www.facebook.com/groups/WaterlooSublet/"
      c) "Housing": "https://www.facebook.com/groups/UWhousing/"  

*Note: The 'Housing' group needs admin's permission to join, so data could not be extracted from this group. :( *

In [1]:
import requests
from IPython.display import HTML
from multiprocessing import Pool
import datetime
import pandas as pd
import numpy as np
import re
import csv
import time
import sys
from geopy.geocoders import GoogleV3
from geopy.distance import vincenty
from geopy.exc import GeocoderTimedOut, GeocoderQuotaExceeded
import gmaps

In [2]:
app_clientID = " " # Input your facebook client id here
app_clientSecret = " " # Input your facebook client secret here
google_API_key = " " # Input your google API key here for the embedded Google Map

In [3]:
housing_groupID_1 = "110354088989367"   # Student Housing in Waterloo
housing_groupID_2 = "142679255268"      # WaterlooSublet
housing_groupID_3 = "198200603621455"   # UWhousing
until_date = datetime.datetime.utcnow().strftime('%Y-%m-%d%H:%M:%S')
limit = 1000

### Generate an app access token

To generate an app access token, we need to make a Graph API call:

                   GET /oauth/access_token
                   ?client_id={app-id}
                   &client_secret={app-secret}
                   &grant_type=client_credentials

More details: https://developers.facebook.com/docs/facebook-login/access-tokens/

In [4]:
def getAccessToken(client_ID, client_secret):
    payload = {
                'client_id': client_ID, 
                'client_secret': client_secret,
                'grant_type': 'client_credentials', 
              }
    
    response = requests.get('https://graph.facebook.com/oauth/access_token?', params = payload)
    assert (response.status_code == 200), "Excepted status code 200, but got status code {} instead".format(response.status_code)

    return response.json()["access_token"]

Check the validility of the returned access token.

In [5]:
def checkReturnedToken(access_token):
    payload = {
                "access_token": access_token
              }

    response = requests.get("https://graph.facebook.com/app?", params=payload)
    assert (response.status_code == 200), "Excepted status code 200, but got status code {} instead".format(response.status_code)

    return response.json()


token = getAccessToken(app_clientID, app_clientSecret)
checkReturnedToken(token)

{'category': 'Education',
 'id': '662210950629125',
 'link': 'https://www.facebook.com/games/?app_id=662210950629125',
 'name': "Harry's Experiment on Housing"}

### Get data from the specified group

In [6]:
def getHousingAds(housing_group_ID, token):
    params = {
                "until": until_date, 
                "limit": str(limit)
             }
    
    headers = { 
                "Authorization": "Bearer {}".format(token)
              }
    
    url = "https://graph.facebook.com/v2.10/{}/feed?".format(housing_group_ID)
    res = requests.get(url, headers=headers, params=params)
    
    return res

In [7]:
housing_ads = {}
housing_ads["data"] = []

for fb_group in [housing_groupID_1, housing_groupID_2, housing_groupID_3]:
    print ("Getting ads from this Fb group ID {}: STATUS".format(fb_group), end=" ")
    res = getHousingAds(fb_group, token)
    print (res.status_code)
    
    if (res.status_code == 200):
        housing_ads["data"].extend(res.json()["data"])

Getting ads from this Fb group ID 110354088989367: STATUS 200
Getting ads from this Fb group ID 142679255268: STATUS 200
Getting ads from this Fb group ID 198200603621455: STATUS 400


# The Housing Post Dataframe

In [8]:
housing = pd.DataFrame(housing_ads["data"]) # Extracting housing data out

housing = housing[["id", "message", "updated_time"]].dropna()
housing

,id,message,updated_time
0,110354088989367_1667760783248682,Housing/ rooms available Today Sept 9. Discou...,2017-09-09T15:35:34+0000
1,110354088989367_1667403016617792,single room near UW campus\n‎ر.س.550‎ - Univer...,2017-09-09T15:12:52+0000
2,110354088989367_1666083753416385,The Hub - Last Few Two Bedroom Suites Remainin...,2017-09-09T14:39:41+0000
3,110354088989367_1664448550246572,LAST 8 MONTH LEASES AVAILABLE - NOW ONLY $625\...,2017-09-09T14:33:14+0000
4,110354088989367_1667704209921006,1 Bedroom in 2-Bedroom Suite (The Hub)\n$750 -...,2017-09-09T14:28:58+0000
5,110354088989367_1667018936656200,TWO MONTHS FREE - 45 COLUMBIA\n$595 - Waterloo...,2017-09-09T13:43:55+0000
6,110354088989367_1667198359971591,Cat and I are looking to not be homeless this ...,2017-09-09T13:35:52+0000
7,110354088989367_1667416063283154,Students housing for 4M to 8M from 2018.\nNT$4...,2017-09-09T06:28:16+0000
8,110354088989367_1665901686767925,Looking for 4 month winter sublet close to UW ...,2017-09-09T05:44:21+0000
9,110354088989367_1667357726622321,Look for a ride from Waterloo to Markham at Sa...,2017-09-09T04:29:33+0000


### Get rid of housing posts that are not supplying rooms for sublet.
Normally, housing posts which have titles **"Looking for ..."**, **"Looking ..."** or **"Parking ..."** are potentially not the housing posts that we want. Also, ignore housing posts that are already sold out (a.k.a no longer available).

In [9]:
def availableRoomPosts(message):
    newline = message.find("\n")
    title = message[:newline].lower()
    if (title.find("looking for") != -1 or title.find("looking") != -1 or title.find("parking") != -1 or title.find("(sold)") != -1):
        if (title.find("?") == -1):
            return False
    
    return True

In [10]:
available_rooms = housing["message"].apply(availableRoomPosts)
rooms = housing[available_rooms]

rooms.reset_index(drop=True, inplace=True)
rooms

,id,message,updated_time
0,110354088989367_1667760783248682,Housing/ rooms available Today Sept 9. Discou...,2017-09-09T15:35:34+0000
1,110354088989367_1667403016617792,single room near UW campus\n‎ر.س.550‎ - Univer...,2017-09-09T15:12:52+0000
2,110354088989367_1666083753416385,The Hub - Last Few Two Bedroom Suites Remainin...,2017-09-09T14:39:41+0000
3,110354088989367_1664448550246572,LAST 8 MONTH LEASES AVAILABLE - NOW ONLY $625\...,2017-09-09T14:33:14+0000
4,110354088989367_1667704209921006,1 Bedroom in 2-Bedroom Suite (The Hub)\n$750 -...,2017-09-09T14:28:58+0000
5,110354088989367_1667018936656200,TWO MONTHS FREE - 45 COLUMBIA\n$595 - Waterloo...,2017-09-09T13:43:55+0000
6,110354088989367_1667416063283154,Students housing for 4M to 8M from 2018.\nNT$4...,2017-09-09T06:28:16+0000
7,110354088989367_1667357726622321,Look for a ride from Waterloo to Markham at Sa...,2017-09-09T04:29:33+0000
8,110354088989367_1665916013433159,"374 Hazel Street\n$560 - Waterloo, Ontario\n\n...",2017-09-09T04:19:04+0000
9,110354088989367_1667118753312885,1 Bedroom in 2-Bedroom Suite (The Hub)\n$750 -...,2017-09-09T02:57:04+0000


Quick check to make sure that we didn't exclude any housing posts that are actually supplying a place to stay.

In [11]:
housing[~available_rooms]

,id,message,updated_time
6,110354088989367_1667198359971591,Cat and I are looking to not be homeless this ...,2017-09-09T13:35:52+0000
8,110354088989367_1665901686767925,Looking for 4 month winter sublet close to UW ...,2017-09-09T05:44:21+0000
20,110354088989367_1661141357243958,"Looking for one roommate\n$400 - Waterloo, Ont...",2017-09-08T21:49:59+0000
24,110354088989367_1667055743319186,Looking for a room to rent for the Fall term. ...,2017-09-08T19:18:05+0000
25,110354088989367_1662696763755084,Looking for Fall Sublet (Female)\nFREE - Water...,2017-09-08T19:11:47+0000
26,110354088989367_1667029083321852,Looking for cat friendly room availible immedi...,2017-09-08T18:34:32+0000
28,110354088989367_1667026446655449,Looking for a last minute fall sublet \n$500\n...,2017-09-08T18:29:33+0000
31,110354088989367_1666938059997621,Looking for a place ASAP. Move in tomorrow\n$4...,2017-09-08T16:27:25+0000
32,110354088989367_1666896576668436,Looking to sublet 4-5 rooms for the 2018 Winte...,2017-09-08T15:35:18+0000
34,110354088989367_1666876223337138,Looking for Parking near 275 Larch Street\n$20...,2017-09-08T15:10:14+0000


### Locate the location of housing using regex
** regex ** is used to find the locations of the housing which are mentioned in the message body. A list of Waterloo street names is used as the regex patterns.

Note: *Attempted to use the NLTK named entity recognition (NER) classifier, which is provided by the Stanford NER tagger, to apply **location tag** for locations mentioned in the message bodies, but to no avail.* :( *Hence, regex is used here.*

Open and load the file that contains a list of street names which have already been preprocessed in the **"Streets-in-Waterloo-Ontario.ipynb"** notebook.

In [12]:
with open('./data/nearby_street.txt', 'r') as f:
    reader = csv.reader(f)
    streets = []
    for r in reader:
        streets.extend(r)

**includePattern** function includes regex patterns for each of the street names. 

The **streets_pattern** variable will be a long string that contains the UNION of the regex patterns of each the street names, with regex pattern as follows:

                            "[0-9]*\s*\b({PATTERN})\b(\s*waterloo)?(\s*,?\s*ontario|on)?"

where  PATTERN = (street1_regex_pattern | street2_regex_pattern | street3_regex_pattern | ..... | streetn_regex_pattern)   

In [13]:
# Online regex patterns tester: https://regex101.com/
def includePattern(streets):
    streets_pattern = []
    
    for street in streets:
        street = street.lower()
        st = street.split()
        
        for idx, ss in enumerate(st[1:]):
            if ss == 'north' or ss == 'south' or ss == 'east' or ss == 'west':
                tmp = r'({0}|{1})'.format(ss, ss[0])
                st[idx+1] = tmp
            elif ss == 'avenue':
                tmp = r'({0}|{1})'.format(ss, 'ave')
                st[idx+1] = tmp
            elif ss == 'street':
                tmp = r'({0}|{1})'.format(ss, 'st')
                st[idx+1] = tmp
        
        s = st[0] + ''.join([r'(\s*{}\b)?'.format(s) for s in st[1:]])
        streets_pattern.append(s)
    
    return streets_pattern

streets_pattern = includePattern(streets) # Include regex patterns for each of the street names
streets_pattern = '|'.join(streets_pattern) # Union each of the street names regex patterns
streets_pattern = r'[0-9]*\s*\b({PATTERN})\b(\s*waterloo)?(\s*,?\s*ontario|on)?'.format(PATTERN=streets_pattern)

Using regex with **streets_pattern** as the regex pattern , **findLocation** function finds the location of the housing that appears in the message body.

In [14]:
def findLocation(txt):
    txt = txt.lower()
    matched = re.search(streets_pattern, txt)
    if matched:
        if matched.group().strip() == 'university': # "University of Waterloo" might be matched by the regex coincidentally
            pos = matched.span()[1]
            if 'of waterloo' in txt[pos:pos+len(' of Waterloo')]:
                return findLocation(txt[pos+len('ofWaterloo')+1: ])
        else:
            return matched.group()
    else:
        return np.nan
    
rooms = rooms.assign(location = rooms['message'].apply(findLocation))
rooms

,id,message,updated_time,location
0,110354088989367_1667760783248682,Housing/ rooms available Today Sept 9. Discou...,2017-09-09T15:35:34+0000,columbia st w
1,110354088989367_1667403016617792,single room near UW campus\n‎ر.س.550‎ - Univer...,2017-09-09T15:12:52+0000,NaN
2,110354088989367_1666083753416385,The Hub - Last Few Two Bedroom Suites Remainin...,2017-09-09T14:39:41+0000,NaN
3,110354088989367_1664448550246572,LAST 8 MONTH LEASES AVAILABLE - NOW ONLY $625\...,2017-09-09T14:33:14+0000,NaN
4,110354088989367_1667704209921006,1 Bedroom in 2-Bedroom Suite (The Hub)\n$750 -...,2017-09-09T14:28:58+0000,9 columbia st w
5,110354088989367_1667018936656200,TWO MONTHS FREE - 45 COLUMBIA\n$595 - Waterloo...,2017-09-09T13:43:55+0000,45 columbia
6,110354088989367_1667416063283154,Students housing for 4M to 8M from 2018.\nNT$4...,2017-09-09T06:28:16+0000,columbia
7,110354088989367_1667357726622321,Look for a ride from Waterloo to Markham at Sa...,2017-09-09T04:29:33+0000,NaN
8,110354088989367_1665916013433159,"374 Hazel Street\n$560 - Waterloo, Ontario\n\n...",2017-09-09T04:19:04+0000,374 hazel street
9,110354088989367_1667118753312885,1 Bedroom in 2-Bedroom Suite (The Hub)\n$750 -...,2017-09-09T02:57:04+0000,9 columbia st w


### For data that we are unable to locate any location using the list of street names, we will try to search if popular condominium' names appear in the message body.
There are several popular condominiums in Waterloo that are favoured by the students:

        i.)   ICON
        ii.)  LUXE
        iii.) The HUB
        iv.)  Blair House
        v.)   Sage
        iv.)  Accommod8u

These are used as metrics to locate the existence of condominiums name in the message body.

In [15]:
def findCondo(txt):
    txt = txt.lower()
    if 'icon' in txt: 
        return 'icon'
    elif 'luxe' in txt: 
        return 'luxe'
    elif 'hub' in txt:
        return 'hub'
    elif 'blair house' in txt:
        return 'blair house'
    elif 'sage' in txt:
        return 'sage'
    elif 'accommod8u'in txt:
        return 'accommod8u'
    
mask = rooms['location'].isnull()
rooms.loc[mask, 'location'] = rooms.loc[mask, 'message'].apply(findCondo)

#### A test to check whether the location of the condominiums (ie. Icon, Luxe, Hub) are located correctly.

In [16]:
condo_mask = []

for r in rooms['location']:
    if r is not np.nan:
        r = str(r)
        condo_mask.append('icon' in r or 'luxe' in r or 'hub' in r or 'blair house' in r or 'sage' in r or 'accommod8u' in r)
    else:
        condo_mask.append(False)

rooms[condo_mask]

,id,message,updated_time,location
2,110354088989367_1666083753416385,The Hub - Last Few Two Bedroom Suites Remainin...,2017-09-09T14:39:41+0000,hub
33,110354088989367_1664452330246194,4/8 months sublet or lease takeover \n$850 - T...,2017-09-08T03:16:01+0000,hub
35,110354088989367_1666288213395939,ICON - sublet available for girl. Starting fro...,2017-09-08T02:13:12+0000,icon
38,110354088989367_1666300516728042,WANTED: 1 Bedroom Unit - Winter 2018\n$1 - Wat...,2017-09-07T23:47:38+0000,sage
50,110354088989367_1654707454554015,LEASE TAKEOVER for 1/2 rooms in LUXE I for Win...,2017-09-07T01:31:10+0000,luxe
73,110354088989367_1661463087211785,"Food \n$50 - Waterloo, Ontario\n\nI offer and ...",2017-09-05T15:04:01+0000,sage
78,110354088989367_1663825463642214,"1 Bedroom in 2 Bedroom Apartment (The Hub), 8 ...",2017-09-05T02:17:58+0000,hub
86,110354088989367_1653033724721388,"The Hub - Last One, Two, and Three Bedroom Sui...",2017-09-04T17:37:44+0000,hub
89,110354088989367_1660310277327066,HOUSING AVAILABLE TODAY BESIDE UNIVERSITY\n$46...,2017-09-04T12:41:26+0000,hub
97,110354088989367_1662554470435980,Subletting 1/5 bedrooms\n$1 - KW4Rent Off-Camp...,2017-09-03T17:05:42+0000,sage


#### Quick check to make sure that data which have locations contain addresses as expected.
It turned out that the regex works pretty good in extracting the location out from the message body.

In [17]:
emptyLocation = rooms['location'].isnull()
rooms[~emptyLocation]

,id,message,updated_time,location
0,110354088989367_1667760783248682,Housing/ rooms available Today Sept 9. Discou...,2017-09-09T15:35:34+0000,columbia st w
2,110354088989367_1666083753416385,The Hub - Last Few Two Bedroom Suites Remainin...,2017-09-09T14:39:41+0000,hub
4,110354088989367_1667704209921006,1 Bedroom in 2-Bedroom Suite (The Hub)\n$750 -...,2017-09-09T14:28:58+0000,9 columbia st w
5,110354088989367_1667018936656200,TWO MONTHS FREE - 45 COLUMBIA\n$595 - Waterloo...,2017-09-09T13:43:55+0000,45 columbia
6,110354088989367_1667416063283154,Students housing for 4M to 8M from 2018.\nNT$4...,2017-09-09T06:28:16+0000,columbia
8,110354088989367_1665916013433159,"374 Hazel Street\n$560 - Waterloo, Ontario\n\n...",2017-09-09T04:19:04+0000,374 hazel street
9,110354088989367_1667118753312885,1 Bedroom in 2-Bedroom Suite (The Hub)\n$750 -...,2017-09-09T02:57:04+0000,9 columbia st w
10,110354088989367_1667158293308931,203 Lester! 1 Unit left! CONTACT me for more i...,2017-09-09T01:44:15+0000,203 lester
12,110354088989367_1667189673305793,subletting a place for 8months from now to Apr...,2017-09-08T23:54:28+0000,333 king street
13,110354088989367_1667113623313398,"Fall 2017 Sublet \n$452 - Waterloo, Ontario\n\...",2017-09-08T23:38:00+0000,hazel


### Extract price
First, extract price from the message body. Then, sort the data according to the price. <br />
**Note**: Some people did not include prices of their rooms or put up a small price (ie. \$1) in their sublet posts as they wanted to know the price that the interested buyers are willing to pay. We are expected to see a lot \$1 or NaN prices in our data.

In [18]:
def findPrice(string):
    # Ignoring decimal points, added constraint for at most three whitespaces between dollar sign and the digits. 
    # Need to set constraint for 'no comma' in the price
    price = re.search(r"\$\s{0,3}\d{1,3}(,\d{3})*", string)
    if price is None:
        return np.nan
    
    return string[price.start():price.end()]
    

def extractPricing(message):
    section = message.split("\n")

    if (len(section) == 1):
        return findPrice(message)

    pricing_slot = section[1]
    price = findPrice(pricing_slot) # find price in the pricing slot
    
    if (price is np.nan):
        return findPrice(message)   # if there is no price in the pricing slot, try to find price in the entire message body
    else :
        return price

In [19]:
rooms = rooms.assign(price = rooms["message"].apply(extractPricing))

# Sort the data
rooms = rooms.assign(price_value = rooms["price"].apply(lambda price: int(price[1:].replace(",", "")) if price is not np.nan else np.nan)) #Ignoring the "$" symbol
rooms.sort_values(["price_value"], inplace=True)
rooms.drop("price_value", axis=1, inplace=True)

# Rearrange the order of the columns
rooms = rooms[["id", "message", "updated_time", "location", "price"]]
rooms.reset_index(drop=True, inplace=True)
rooms

,id,message,updated_time,location,price
0,142679255268_10155430115640269,4 or 8 month (Jan-Aug) sublet at 316 King\n$1 ...,2017-09-08T23:17:45+0000,316 king,$1
1,110354088989367_1666122873412473,4/8 Month Sublet / Lease Takeover from January...,2017-09-08T05:42:17+0000,228 albert,$1
2,142679255268_10155321327585269,"Storage (Aug) a week, a month \n$1 - Waterloo,...",2017-07-31T03:19:05+0000,sage,$1
3,110354088989367_1657368857621208,Single Room @ Velocity Residence (Fall 2017)\n...,2017-09-01T08:39:25+0000,residence,$1
4,110354088989367_1666300516728042,WANTED: 1 Bedroom Unit - Winter 2018\n$1 - Wat...,2017-09-07T23:47:38+0000,sage,$1
5,110354088989367_1666296250061802,Subletting 1/5 bedrooms\n$1 - KW4Rent Off-Camp...,2017-09-07T23:39:21+0000,251 lester st,$1
6,110354088989367_1631689093522518,"FURNITURE\n$1 - Waterloo, Ontario\n\n***FURNIT...",2017-08-03T01:47:24+0000,None,$1
7,142679255268_10155338592960269,I need a sublet for september 1st\n$1 - Waterl...,2017-08-07T14:23:17+0000,sage,$1
8,142679255268_10155351331465269,Many items in really low price\n$1 - Victoria ...,2017-08-10T21:11:32+0000,None,$1
9,110354088989367_1628924307132330,"1 month storage (Aug) \n$1 - Waterloo, Ontario...",2017-07-31T03:19:08+0000,sage,$1


### Find the total number of comments for each housing post.
Normally, people who are interested in a particular sale post in Facebook would drop a comment at the comment section of that post. <br /> 
-> If a sale post has a great amount of comments, intuitively we know that everyone favours the item of that sale posts. <br />
-> By contrast, if a sale post has few comments, intuitively we know that only minority favours the item. <br /> 
However, note that the total comments of a sale post does not entirely determine the item's popularity. For simplicity, here we just take into consideration of the total comments of a sale post in determining the item's popularity.

By using the total count of comments for each housing post, we could try to understand what location and price range are favoured by the majority.

In [20]:
def total_comments(post_id):
    headers = { 
                "Authorization": "Bearer {}".format(token)
              }
    
    url = "https://graph.facebook.com/v2.10/{}/comments?summary=1".format(post_id)
    res = requests.get(url, headers=headers)
    
    assert res.status_code == 200, "Could not get a summary of comments from this postId {}".format(post_id)
    
    total_count = res.json()["summary"]["total_count"]
    
    comments = {
                "id": post_id,
                "total comments": total_count
               }
    
    return comments

Here, we utilize the benefit of **multiprocessing**. There is a **10x** speed faster using multiprocessing.

In [21]:
def get_comment_summary(post_ids, token, multiprocessing=False, process_num=20, size=None):
    ids = post_ids[:size]
    
    if multiprocessing:
        with Pool(processes=process_num) as p:
            list_dicts = p.map(total_comments, ids)
            p.terminate()
            p.join()        
    else:
        list_dicts = [total_comments(idd) for idd in ids]
    
    return list_dicts

#### Experiment on 100 id's with and without multiprocessing

In [22]:
post_ids = rooms["id"].tolist()

test_size = 100

tic = time.time()
comments = get_comment_summary(post_ids, token, size=test_size)
toc = time.time()
print("Total execution time on {} id's WITHOUT multiprocessing: {}s".format(test_size, toc - tic))

tic = time.time()
comments = get_comment_summary(post_ids, token, multiprocessing=True, size=test_size)
toc = time.time()
print("Total execution time on {} id's WITH multiprocessing:  {}s".format(test_size, toc - tic))

Total execution time on 100 id's WITHOUT multiprocessing: 21.674668073654175s
Total execution time on 100 id's WITH multiprocessing:  4.719592809677124s


#### Get comments for ALL id's with multiprocessing

In [23]:
tic = time.time()
comments = get_comment_summary(post_ids, token, multiprocessing=True)
toc = time.time()
print("Total execution time on ALL {} id's WITH multiprocessing:  {}s".format(len(post_ids), toc - tic))

Total execution time on ALL 689 id's WITH multiprocessing:  16.627751111984253s


#### Merge rooms dataFrame with the comments

In [24]:
rooms = rooms.merge(pd.DataFrame(comments), how="left") # left join
rooms

,id,message,updated_time,location,price,total comments
0,142679255268_10155430115640269,4 or 8 month (Jan-Aug) sublet at 316 King\n$1 ...,2017-09-08T23:17:45+0000,316 king,$1,0
1,110354088989367_1666122873412473,4/8 Month Sublet / Lease Takeover from January...,2017-09-08T05:42:17+0000,228 albert,$1,3
2,142679255268_10155321327585269,"Storage (Aug) a week, a month \n$1 - Waterloo,...",2017-07-31T03:19:05+0000,sage,$1,0
3,110354088989367_1657368857621208,Single Room @ Velocity Residence (Fall 2017)\n...,2017-09-01T08:39:25+0000,residence,$1,8
4,110354088989367_1666300516728042,WANTED: 1 Bedroom Unit - Winter 2018\n$1 - Wat...,2017-09-07T23:47:38+0000,sage,$1,0
5,110354088989367_1666296250061802,Subletting 1/5 bedrooms\n$1 - KW4Rent Off-Camp...,2017-09-07T23:39:21+0000,251 lester st,$1,0
6,110354088989367_1631689093522518,"FURNITURE\n$1 - Waterloo, Ontario\n\n***FURNIT...",2017-08-03T01:47:24+0000,None,$1,0
7,142679255268_10155338592960269,I need a sublet for september 1st\n$1 - Waterl...,2017-08-07T14:23:17+0000,sage,$1,1
8,142679255268_10155351331465269,Many items in really low price\n$1 - Victoria ...,2017-08-10T21:11:32+0000,None,$1,0
9,110354088989367_1628924307132330,"1 month storage (Aug) \n$1 - Waterloo, Ontario...",2017-07-31T03:19:08+0000,sage,$1,0


# Which location is favoured by majority?
This section tries to understand which location is favoured by the majority by plotting out the top 30 places with the highest comment.

In [25]:
favoured_rooms = rooms.sort_values(["total comments"], ascending=False) # Sort according to 'total comments' in descending order

top_30_rooms = favoured_rooms.head(30)  # top 30 places that are favoured by the majority
least_favoured_rooms = favoured_rooms.tail(-30) # The remaining places that are less favoured by the majority

top_30_rooms

,id,message,updated_time,location,price,total comments
393,110354088989367_1598871360137625,Fall 2017 4-Month Sublet at the HUB\n$650 - Un...,2017-09-04T17:38:36+0000,columbia,$650,80
244,142679255268_10155258325455269,"2/2 Rooms For FALL TERM 2017\n$550 - Waterloo,...",2017-08-14T04:39:03+0000,university ave,$550,40
330,142679255268_10155386887345269,FALL 2017 SUBLET @ 168 King Street North\n$615...,2017-08-30T21:24:35+0000,168 king street,$615,35
647,142679255268_10155260206715269,"208 Sunview St., 11th Floor, Room available fo...",2017-08-02T01:43:23+0000,208 sunview st,NaN,35
296,110354088989367_1465900146768081,ENSUITE BATHROOM -28 Univerisity Street $595! ...,2017-07-29T17:26:22+0000,None,$595,34
265,110354088989367_1627560277268733,"SUBLET AVAILABLE SEPT-MAY \n$577 - Kitchener, ...",2017-08-10T13:51:03+0000,173 king street,$577,33
441,110354088989367_1652775241413903,Sublet @ Icon\n$700 - Phillip And Columbia\n\n...,2017-08-28T15:06:17+0000,phillip,$700,32
629,142679255268_10155389334380269,"208 Sunview St., 11th Floor, Room available fo...",2017-08-30T21:23:52+0000,208 sunview st,NaN,31
631,142679255268_10155346661365269,[NOT TAKING ANY MORE REQUESTS]\n\nSubletting 1...,2017-08-29T07:50:07+0000,181 lester st,NaN,31
675,142679255268_10153930439010269,"Hello everyone, thanks for using my app I hope...",2017-07-02T22:17:46+0000,sage,NaN,31


**calculate_coordinate** function returns the coordinate of the address.

In [ ]:
def calculate_coordinate(id_loc):
    geolocator = GoogleV3() # Google Maps API
    
    idd = id_loc[0]
    address = id_loc[1]
    
    coord_dict = {
                        "id" :        idd,
                        "location":   address,
                        "coordinate": np.nan,
                        "distance (km)":   np.nan
                 }
    
    if address is np.nan or address is None:
        return coord_dict
    
    try:
        UW_coordinates = (43.469757, -80.5409518)
        complete_address = "{}, waterloo, ontario, canada".format(address)
        housing = geolocator.geocode(complete_address, timeout=10)
        housing_coordinates = (housing.latitude, housing.longitude)
        
        coord_dict["coordinate"] = housing_coordinates
        coord_dict["distance (km)"] = round(vincenty(housing_coordinates, UW_coordinates).kilometers, 3)
        return coord_dict
    
    except GeocoderTimedOut:
        return calculate_coordinate(id_loc)
    
    except GeocoderQuotaExceeded:
        # Google Maps (standard API) only allows:
        #     1.) 2,500 free requests per day, calculated as the sum of client-side and server-side queries.
        #     2.) 50 requests per second, calculated as the sum of client-side and server-side queries.
        return calculate_coordinate(id_loc)
    
    except:
        print ("Unexpected error:", sys.exc_info()[0])  # Hack to check the type of exception thrown by Geocoder
        return "Error in finding the coordinates for address: %s" % address

Again, we utilize the benefit of **multiprocessing** which is significantly faster!

In [ ]:
def get_coordinates(loc_list):
    with Pool(processes=20) as p:
        coord_list = p.map(calculate_coordinate, loc_list)
        p.terminate()
        p.join()        
    
    return coord_list

top30_list = top_30_rooms[["id", "location"]].values.tolist()
rest_list = least_favoured_rooms[["id", "location"]].values.tolist()

top_coord_location = get_coordinates(top30_list)
bottom_coord_location = get_coordinates(rest_list)

Unexpected error: <class 'KeyboardInterrupt'>
Unexpected error: <class 'KeyboardInterrupt'>
Unexpected error: <class 'KeyboardInterrupt'>
Unexpected error: <class 'KeyboardInterrupt'>
Unexpected error: <class 'KeyboardInterrupt'>
Unexpected error: <class 'KeyboardInterrupt'>
Unexpected error: <class 'KeyboardInterrupt'>
Unexpected error: <class 'KeyboardInterrupt'>
Unexpected error: <class 'KeyboardInterrupt'>
Unexpected error: <class 'KeyboardInterrupt'>
Unexpected error: <class 'KeyboardInterrupt'>
Unexpected error: <class 'KeyboardInterrupt'>


Extract all coordicates from the list of dict. Also, drop data that have *NaN* location (i.e. No location found for that data)

In [ ]:
top_coord_list = [tcl["coordinate"] for tcl in top_coord_location if isinstance(tcl["coordinate"], tuple) == True]
bottom_coord_list = [bcl["coordinate"] for bcl in bottom_coord_location if isinstance(bcl["coordinate"], tuple) == True]

### Embedding Google Maps in Jupyter Notebook
#### For better understanding and visualization of the popular places favoured by the majority, we embed Google Maps here using gmaps, a Jupyter plugin.

A marker is plotted in the embedded Google Maps to indicate the location of the University of Waterloo. <br />

Circle symbols, each represents the latitude and longitude pair of the places, are plotted in the map as well. <br />
-> **Crimson** cicle symbol represents the top 10 locations favoured by the people. <br />
-> **Indigo** circle symbol represents the next top 10 locations favoured by the people. <br />
-> **Lime Green** circle symbol represents the following top 10 locations favoured by the people. <br />
-> **Violet** circle symbol represents the remaining locations that are less favoured by the people.

*Documentation for **gmaps**: http://jupyter-gmaps.readthedocs.io/en/latest/gmaps.html*

In [ ]:
# http://jupyter-gmaps.readthedocs.io/en/latest/gmaps.html
gmaps.configure(api_key=google_API_key)
top_10_locations = top_coord_list[:10]
top_20_locations = top_coord_list[10:20]
top_30_locations = top_coord_list[20:]

uwaterloo_coordinate = (43.4723, -80.5449)
fig = gmaps.figure(center=uwaterloo_coordinate, zoom_level=14)
uwaterloo_layer = gmaps.marker_layer([uwaterloo_coordinate])
fig.add_layer(uwaterloo_layer)
### Not so popular location
rooms_layer = gmaps.symbol_layer(bottom_coord_list, fill_color="#EE82EE", stroke_color="#EE82EE", scale=2)  # violet color
fig.add_layer(rooms_layer)
### top 30 locations
rooms_layer = gmaps.symbol_layer(top_10_locations, fill_color="#DC143C", stroke_color="#DC143C", scale=4)   # crimson color
fig.add_layer(rooms_layer)
rooms_layer = gmaps.symbol_layer(top_20_locations, fill_color="#4B0082", stroke_color="#4B0082", scale=4)   # indigo color
fig.add_layer(rooms_layer)
rooms_layer = gmaps.symbol_layer(top_30_locations, fill_color="#32CD32", stroke_color="#32CD32", scale=4)   # lime green color
fig.add_layer(rooms_layer)

fig

Using left join, merge the **favoured_rooms** dataframe with the new **coordinates** and **locations** columns.

In [ ]:
top_coord_location.extend(bottom_coord_location)
merged_rooms = pd.merge(favoured_rooms, pd.DataFrame(top_coord_location), how="left", on=["id", "location"])

merged_rooms = merged_rooms[["id", "message", "updated_time", "location", "distance (km)", "price", "total comments"]]

## Render the DataFrame as HTML table for faster access to the housing post in Facebook.

In [ ]:
merged_rooms = merged_rooms.assign(link = merged_rooms["id"].apply(lambda id: '<a href="https://www.facebook.com/groups/{0}/permalink/{1}">link</a>'
                                              .format(id.split('_')[0], id.split('_')[1])))

pd.set_option('display.max_colwidth', 100)  # Show the html links in the dataframe table
rooms_with_Url = HTML(merged_rooms.to_html(escape=False))

rooms_with_Url

## Write the dataframe to a comma-separated values (csv) file for further experiment.

In [ ]:
merged_rooms.to_csv(path_or_buf="./data/facebook_sublet.csv", index=False)